In [ ]:
from raiwidgets import FairnessDashboard
from sklearn.model_selection import train_test_split

from sklearn import svm
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
import pandas as pd

In [ ]:
from sklearn.datasets import fetch_openml
data = fetch_openml(data_id=1590, as_frame=True)
X_raw = data.data
Y = (data.target == '>50K') * 1
X_raw

In [ ]:
A = X_raw[['sex','race']]
X = X_raw.drop(labels=['sex', 'race'],axis = 1)
X = pd.get_dummies(X)

sc = StandardScaler()
X_scaled = sc.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

le = LabelEncoder()
Y = le.fit_transform(Y)


X_train,\
    X_test,\
    Y_train,\
    Y_test,\
    A_train,\
    A_test = train_test_split(X_scaled,
                              Y,
                              A,
                              test_size=0.2,
                              random_state=0,
                              stratify=Y)


X_train = X_train.reset_index(drop=True)
A_train = A_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
A_test = A_test.reset_index(drop=True)


In [ ]:
lr_predictor = LogisticRegression(
    solver='liblinear', fit_intercept=True)

lr_predictor.fit(X_train, Y_train)

#svm_predictor = svm.SVC()

#svm_predictor.fit(X_train, Y_train)

In [ ]:
lr_predictor.predict(X_test)

In [ ]:
FairnessDashboard(sensitive_features=A_test,
                  y_true=Y_test,
                  y_pred=lr_predictor.predict(X_test)
)

In [ ]:
svm_predictor = svm.SVC()

svm_predictor.fit(X_train, Y_train)

In [ ]:
FairnessDashboard(sensitive_features=A_test,
                  y_true=Y_test,
                  y_pred={
                      "SVM":svm_predictor.predict(X_test)
                  })

In [ ]:
FairnessDashboard(sensitive_features=A_test,
                  y_true=Y_test,
                  y_pred={
                      "Log. Reg.":lr_predictor.predict(X_test),
                      "SVM":svm_predictor.predict(X_test)
                  })